In [27]:
# Opracowane na podstawie https://minpy.readthedocs.io/en/latest/tutorial/rnn_mnist.html
using MLDatasets, Flux
train_data = MLDatasets.MNIST(split=:train)
test_data  = MLDatasets.MNIST(split=:test)

function loader(data; batchsize::Int=1)
    x1dim = reshape(data.features, 28 * 28, :) # reshape 28×28 pixels into a vector of pixels
    yhot  = Flux.onehotbatch(data.targets, 0:9) # make a 10×60000 OneHotMatrix
    Flux.DataLoader((x1dim, yhot); batchsize, shuffle=true)
end

net = Chain(
    RNN((14 * 14) => 64, tanh),
    Dense(64 => 10, identity),
)

Chain(
  Recur(
    RNNCell(196 => 64, tanh),           # 16_768 parameters
  ),
  Dense(64 => 10),                      # 650 parameters
)         # Total: 6 trainable arrays, 17_418 parameters,
          # plus 1 non-trainable, 64 parameters, summarysize 68.406 KiB.

In [28]:
using Statistics: mean  # standard library
function loss_and_accuracy(model, data)
    (x,y) = only(loader(data; batchsize=length(data)))
    Flux.reset!(model)
    ŷ = model(x[  1:196,:])
    ŷ = model(x[197:392,:])
    ŷ = model(x[393:588,:])
    ŷ = model(x[589:end,:])
    loss = Flux.logitcrossentropy(ŷ, y)  # did not include softmax in the model
    acc = round(100 * mean(Flux.onecold(ŷ) .== Flux.onecold(y)); digits=2)
    (; loss, acc, split=data.split)  # return a NamedTuple
end

@show loss_and_accuracy(net, test_data);  # accuracy about 10%, before training

train_log = []
settings = (;
    eta = 15e-3,
    epochs = 5,
    batchsize = 100,
)

opt_state = Flux.setup(Descent(settings.eta), net);

loss_and_accuracy(net, test_data) = (loss = 2.5960243f0, acc = 8.38, split = :test)


In [31]:
using ProgressMeter

for epoch in 1:settings.epochs
    @time for (x,y) in loader(train_data, batchsize=settings.batchsize)
        Flux.reset!(net)
        grads = Flux.gradient(model -> let
                ŷ = model(x[  1:196,:])
                ŷ = model(x[197:392,:])
                ŷ = model(x[393:588,:])
                ŷ = model(x[589:end,:])
                Flux.logitcrossentropy(ŷ, y)
            end, net)
        Flux.update!(opt_state, net, grads[1])
    end
    
    loss, acc, _ = loss_and_accuracy(net, train_data)
    test_loss, test_acc, _ = loss_and_accuracy(net, test_data)
    @info epoch acc test_acc
    nt = (; epoch, loss, acc, test_loss, test_acc) 
    push!(train_log, nt)
end

  6.067303 seconds (829.05 k allocations: 2.649 GiB, 15.57% gc time, 16.97% compilation time)


┌ Info: 1
│   acc = 94.94
└   test_acc = 94.65


  5.506500 seconds (536.54 k allocations: 2.633 GiB, 5.63% gc time)


┌ Info: 2
│   acc = 95.34
└   test_acc = 94.94


  5.992735 seconds (536.54 k allocations: 2.633 GiB, 15.73% gc time)


┌ Info: 3
│   acc = 95.68
└   test_acc = 95.01


  4.804795 seconds (536.54 k allocations: 2.633 GiB, 6.50% gc time)


┌ Info: 4
│   acc = 95.93
└   test_acc = 95.44


  4.682319 seconds (536.54 k allocations: 2.633 GiB, 6.96% gc time)


┌ Info: 5
│   acc = 96.21
└   test_acc = 95.6


In [32]:
Flux.reset!(net)
x1, y1 = first(loader(train_data)); # (28×28×1×1 Array{Float32, 3}, 10×1 OneHotMatrix(::Vector{UInt32}))
y1hat = net(x1[  1:196,:])
y2hat = net(x1[197:392,:])
y3hat = net(x1[393:588,:])
y4hat = net(x1[589:end,:])
@show hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9))

@show loss_and_accuracy(net, train_data);

hcat(Flux.onecold(y1hat, 0:9), Flux.onecold(y1, 0:9)) = [9 3]
loss_and_accuracy(net, train_data) = (loss = 0.13418642f0, acc = 96.21, split = :train)


In [19]:
ŷ

LoadError: UndefVarError: `ŷ` not defined

In [18]:
y2hat

10×1 Matrix{Float32}:
 -1.5439503
  0.32302755
  2.0229397
 -0.14970115
 -2.5560298
  4.495308
 -2.9564798
  2.5527892
 -0.7681055
  1.46721

In [9]:
y1hat

10×1 Matrix{Float32}:
  7.5319905
 -2.5360425
  1.3435442
  1.7613591
 -3.7410812
  3.890618
 -2.5021336
 -2.069571
  1.9236335
 -0.4906675